In [1]:
import dotenv
import os
import sys
sys.setrecursionlimit(1000)
dotenv.load_dotenv(".env", override=True)

True

In [2]:
from lang_memgpt import (
    memgraph
)

/home/ubuntu/anaconda3/envs/langchain/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## create pinecone

In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

In [4]:
index_name = os.environ["PINECONE_INDEX_NAME"]
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name="lang-memgpt",
        dimension=1024, # Replace with your model dimensions
        metric="cosine", # Replace with your model metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )

In [5]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm_gpt = ChatOpenAI(model="gpt-4o",
                  temperature = 0.8,
                max_tokens = 4096,
                    # stop_sequences = ['</invoke>','</error>'] ,
                  streaming=True
                )

In [6]:
from langchain_aws import ChatBedrockConverse
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_sonnet = ChatBedrockConverse(model="anthropic.claude-3-5-sonnet-20240620-v1:0",
                                 credentials_profile_name='c35',
                  temperature = 0.8,
                max_tokens = 4096,
                    stop_sequences = ['</invoke>','</error>'] 
                )

/home/ubuntu/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The class `ChatBedrockConverse` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [7]:
import uuid

user_id = 'user3'
thread_id = str(uuid.uuid4())  # can adjust

In [43]:
class Chat:
    def __init__(self, user_id: str, thread_id: str):
        self.thread_id = thread_id
        self.user_id = user_id

    async def __call__(self, messages: list) -> str:
        chunks = memgraph.astream_events(
            input={
                "messages": messages,
            },
            config={
                "configurable": {
                    "llm":llm_gpt,
                    "user_id": self.user_id,
                    "thread_id": self.thread_id,
                }
            },
            version="v2",
        )
        res = ""
        flag= True
        async for event in chunks:
            # print(event)
            if event.get("event") == "on_chat_model_stream":
                tok = event["data"]["chunk"].content
                if tok:
                    if flag:
                        print("\n-------answer------:\n")
                        flag = False
                    print (tok,end="")
                    res += tok
                    # if tok.get('type') == 'tool_use':
                    #     continue
                    #     # if tok.get('name'):
                    #     #     print(tok.get('name'),end="")
                    #     # elif tok.get('input'):
                    #     #     print(tok.get('input'),end="")
                    # elif tok.get('type') == 'text':
                    #     if flag:
                    #         print("\n-------answer------")
                    #         flag = False
                    #     print(tok.get('text'),end="")
                    #     res += tok.get('text')
        return res
    
    # def __call__(self, messages: list) -> str:
    #     final_state = memgraph.invoke(
    #         input={
    #             "messages": messages,
    #         },
    #         config={
    #             "configurable": {
    #                 "llm":llm_gpt,
    #                 "user_id": self.user_id,
    #                 "thread_id": self.thread_id,
    #             }
    #         }
    #     )
    #     return final_state["messages"][-1].content

In [58]:
messages= []
async def handle_chat(input:str,chat:Chat):
    messages.append(('user',input))
    resp = await chat(messages)
    messages.append(('assistant',resp))
    return resp
    

In [59]:
chat = Chat(user_id, thread_id)

In [60]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio
import nest_asyncio

In [67]:
# 启用嵌套事件循环
nest_asyncio.apply()
text_input = widgets.Text(description='输入:')
send_button = widgets.Button(description='发送')

# 创建输出区域
output = widgets.Output()

# 定义发送按钮点击事件的处理函数
def on_send_button_clicked(b):
    async def process():
        with output:
            clear_output()
            print('ssss')
            response = await handle_chat(text_input.value,chat)
            print(response)
        text_input.value = ''  
    # 使用 asyncio.create_task 来运行异步函数
    with output:
        asyncio.create_task(process())

# 将点击事件与处理函数关联
send_button.on_click(on_send_button_clicked)

# 显示聊天界面
display(widgets.VBox([widgets.HBox([text_input, send_button]), output]))

In [64]:
res = await handle_chat('你好',chat)

----search_memory----:Human: 你好
AI: 你好，老朽乃半仙大师。不知阁下有何指教？
Human: 你好


----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']

-------answer------:

你好，老朽在此。不知阁下有何疑问或烦忧需要解答？命里有时终须有，命里无时莫强求。

In [65]:
res = await handle_chat('算一下我的猫',chat)

----search_memory----:Human: 你好
AI: 你好，老朽乃半仙大师。不知阁下有何指教？
Human: 你好
AI: 你好，老朽在此。不知阁下有何疑问或烦忧需要解答？命里有时终须有，命里无时莫强求。
Human: 算一下我的猫


----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']

-------answer------:

哦，阁下的猫儿是位特别的朋友呢。既然如此，请告知它的姓名及出生年月日吧，老朽好为它算上一算。

In [66]:
res

'哦，阁下的猫儿是位特别的朋友呢。既然如此，请告知它的姓名及出生年月日吧，老朽好为它算上一算。'

In [70]:
_ = await chat("我有一只猫.")

----search_memory----:Human: 我有一只猫.


----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']

-------answer------:

你的猫是只非常特别的中国狸花猫，出生在紫禁城里，对吗？你最喜欢和它一起做什么呢？

In [71]:
_ = await chat("是一只狸花猫，它是出生在故宫里.")

----search_memory----:Human: 是一只狸花猫，它是出生在故宫里.


----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']
----save_recall_memory----:User has a Chinese Li Hua cat that was born in the Forbidden City.

-------answer------:

那真是特别的经历！你的狸花猫能在故宫那样的历史环境中出生，真是非常独特。你平时会给它讲有关故宫的故事吗？

In [72]:
_ = await chat("稍等，我要接个电话.")

----search_memory----:Human: 稍等，我要接个电话.


----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']

-------answer------:

好的，你忙你的，我在这等你。

In [73]:
_ = await chat("回来了，我们继续聊吧，刚才聊到哪儿了？")

----search_memory----:Human: 回来了，我们继续聊吧，刚才聊到哪儿了？


----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']

-------answer------:

你好，欢迎回来！我们上次聊到你那只可爱的中国狸花猫，它是出生在紫禁城的。你平时喜欢小动物，今天想聊点什么呢？

In [74]:
_ = await chat("你觉得我的猫的品种怎么样")

----search_memory----:Human: 你觉得我的猫的品种怎么样


----load_memories----
core_memories: []
recall_memories: ['User likes small animals.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.']

-------answer------:

听起来你的猫非常特别，作为一只中国狸花猫，它有着独特的历史和文化背景，尤其是它出生在故宫里，这更增添了它的神秘和高贵。中国狸花猫以其机灵、好动和独立的性格而闻名，同时它们也非常忠诚和友好。你平时喜欢和它一起做些什么呢？

In [75]:
_ = await chat("我喜欢小动物")

----search_memory----:Human: 我喜欢小动物


----load_memories----
core_memories: []
recall_memories: ['User likes small animals.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.']

-------answer------:

小动物确实很可爱！你提到过你有一只在紫禁城出生的中国狸花猫。你还有其他的宠物吗？

In [76]:
_ = await chat("呃呃，我今天想看电影")

----search_memory----:Human: 呃呃，我今天想看电影


----load_memories----
core_memories: []
recall_memories: ['User likes small animals.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.']

-------answer------:

哇，好主意！你有特别想看的电影类型吗？例如，动作片、爱情片、科幻片，或者其他类型？还是说你有具体的电影已经在考虑呢？

In [77]:
_ = await chat("跟我的爱好相关的")

----search_memory----:Human: 跟我的爱好相关的
----load_memories----
core_memories: []
recall_memories: ['User likes small animals.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.']
----search_memory----:User's hobbies

-------answer------:

目前我知道你喜欢小动物，而且你有一只在紫禁城出生的中华狸花猫。你还有其他的爱好吗？或者你愿意分享更多关于你的小动物的故事吗？

In [78]:
_ = await chat("推荐符合我的口味的电影")

----search_memory----:Human: 推荐符合我的口味的电影
----load_memories----
core_memories: []
recall_memories: ['User likes small animals.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.']
----search_memory----:user's taste in movies

-------answer------:

由于我目前没有具体关于你电影品味的记忆，可以告诉我你喜欢什么样的电影类型吗？这样我可以更好地为你推荐符合你口味的电影。

In [79]:
_ = await chat("科幻片")

----search_memory----:Human: 科幻片
----load_memories----
core_memories: []
recall_memories: ['User likes small animals.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.']

-------answer------:

你喜欢看科幻片吗？最近有看过什么让你印象深刻的科幻电影吗？

In [80]:
_ = await chat("星际穿越")

----search_memory----:Human: 星际穿越
----load_memories----
core_memories: []
recall_memories: ['User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User has a Chinese Li Hua cat that was born in the Forbidden City.', 'User likes small animals.']

-------answer------:

《星际穿越》（Interstellar）是一部由克里斯托弗·诺兰执导的科幻电影，于2014年上映。这部电影融合了复杂的科学概念和深刻的情感故事，讲述了一群宇航员穿越虫洞以寻找人类新家园的冒险旅程。

你对《星际穿越》有特别的兴趣吗？例如，喜欢它的剧情、特效，还是其中的科学理念？